In [1]:
import ape
from ape.api import ProviderAPI, BlockAPI, TransactionAPI
import pandas as pd
from darkmode import darkmode_orange  # pylint: disable=unused-import
from ape import Contract
from ape.contracts import ContractInstance
from ape.contracts.base import ContractTransaction, ContractTransactionHandler

/home/mihai/.pyenv/versions/3.9.16/envs/APE-934/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 ## Set up ape

In [2]:
USE_ALCHEMY = False
PROVIDER_STRING = "alchemy" if USE_ALCHEMY else "http://localhost:8547"
provider: ProviderAPI = ape.networks.parse_network_choice(f"ethereum:goerli:{PROVIDER_STRING}").push_provider()

INFO: Connecting to existing Erigon node at  http://localhost:8547.


 ### show `block.base_fee` behavior is unchanged

In [3]:
block = ape.chain.blocks[-1]
block2 = provider.get_block("latest")
assert block == block2
print(f"block.base_fee  = {getattr(block, 'base_fee'):,.0f}")
print(f"block2.base_fee = {getattr(block2, 'base_fee'):,.0f}")

block.base_fee  = 1,186,909,026,049
block2.base_fee = 1,186,909,026,049


 ### show new `provider.base_fee` behavior

In [7]:
fee_history = provider.web3.eth.fee_history(1, provider.web3.eth.get_block("latest").number)
last_base_fee, next_base_fee = fee_history["baseFeePerGas"]
print(f"current base fee is         {last_base_fee/1e9:,.2f} 😴")
print(f"base fee FROM THE FUTURE is {next_base_fee/1e9:,.2f} 🤪")
print(f"provider's base fee  is...  {getattr(provider, 'base_fee')/1e9:,.2f} 🚀")

current base fee is         1,118.58 😴
base fee FROM THE FUTURE is 1,242.69 🤪
provider's base fee  is...  1,242.69 🚀


### preparing a new transaction uses the new base fee

In [8]:
zero_address = "0x0000000000000000000000000000000000000000"
dai: ContractInstance = Contract("0x11fe4b6ae13d2a6055c8d9cf65c55bac32b5d844")  # goerli DAI
contract_txn: ContractTransactionHandler = dai.approve
contract_txn = ContractTransaction(address=dai.address,abi=dai.approve.abis[0])
serial_txn: TransactionAPI = contract_txn.serialize_transaction(zero_address, 1, gas_limit=int(1e6))
prepped_txn: TransactionAPI = provider.prepare_transaction(serial_txn)
max_fee = prepped_txn.max_fee
max_priority_fee = prepped_txn.max_priority_fee
implied_base_fee = max_fee - max_priority_fee
print(f"base fee = {implied_base_fee/1e9:,.2f}")
print(f"         = max_fee ({max_fee/1e9:,.2f}) - max_priority_fee ({max_priority_fee/1e9:,.2f})")

base fee = 1,242.69 (nice.)
         = max_fee (1,243.57) - max_priority_fee (0.88)
